In [ ]:
#!pip install sentinelhub

This notebook show how to use the odc_sh engine in order to load the Sentinel data using SentinelHUB.

In [ ]:
import os 
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import datetime
import cartopy.crs as ccrs
from sentinelhub import DataCollection
import pandas

%matplotlib inline

from odc_sh import engine

os.environ["SH_CLIENT_ID"] = sh_client_id
os.environ["SH_CLIENT_SECRET"] = sh_client_secret

Load data from Sentinel HUB. For other collections check the DataCollection or examples folder.

List all available collections (in some cases additional filter is needed in order to differenciate between the collection ids - e. g. DataCollection.SENTINEL1)

Initialize datacube engine

In [ ]:
dc = engine.Datacube()

In [ ]:
products = dc.list_sh_products()
products

In [ ]:
bands = dc.list_sh_bands()
bands

In [ ]:
def dataframe_for_band(bands):
    band_info = pandas.DataFrame(bands)
    prod_name = np.empty(len(band_info), dtype=object)
    prod_name[:]=pr.name
    band_info.insert(0, "collection", prod_name)
    return band_info

In [ ]:
        bands_info = []
        for pr in dc.get_available_collections():
            if not any(bands_info):
                bands_info = dataframe_for_band(pr.bands)
            else:
                bands_info = pandas.concat([bands_info, dataframe_for_band(pr.bands)]) 
        bands_info


In [ ]:
col = dc.get_collection("SENTINEL2_L1C")
col.bands

In [ ]:
cols = ['bands']

import pandas
for pr in  dc.get_available_collections():
    print(pr.name)
    print(pandas.DataFrame.from_dict(pr.bands ))
    print(pandas.DataFrame.from_dict(pr.metabands))
    print("_____________________")

#rows = [[getattr(pr, col, None) for col in cols] for pr in DataCollection.get_available_collections()]
#pd = pandas.DataFrame(rows, columns=cols).set_index('name', drop=False)

Load Sentinel 1 IW data. Additional filter with polarization needs to be specified. You can also limit the data by orbit.

In [ ]:
resolution = 100  # in meters
longitude = (11.987527, 12.004914)
latitude = (41.990797, 42.018348)

time = ("2019-01-01", "2019-01-04")

ds = dc.load(
    product=DataCollection.SENTINEL2_L1C,
    latitude=latitude,
    longitude=longitude,
    time=time,
    sh_resolution=resolution,
    sh_client_id="6d4d39fa-bc22-4a26-b4e5-60abadaac0ac",
    sh_client_secret="50fd0886-c54b-4c52-8",
  #  sh_filter={"swath_mode": "IW", "orbit_direction": "DESCENDING"},
)

Display properties of the x-array datacube

Store downloaded data to .nc format

In [ ]:
#ds.to_netcdf("s2_all_bands.nc")

You can load the data 

In [ ]:
#ds_stored = xr.open_dataset("s2_all_bands.nc")
#ds_stored

Dislay available time intervals

In [ ]:
for t in ds.time:
    print(f"times: {t.data}")

Get first available frame

In [ ]:
single_time_frame = ds.sel(time=ds.time[0], bands=['B04','B03','B02'])
#single_time_frame

In [ ]:
crs = ccrs.UTM("18")
fig, ax = plt.subplots(nrows=1, ncols=len(ds.time), figsize=(12, 4), dpi=80)

for idx, t in enumerate(ds.time):
    print(f"time: {t.data}")
    single_time_frame = ds.sel(time=ds.time[idx], bands=['B04','B03','B02'], drop=True)
    single_time_frame= single_time_frame / single_time_frame.data.max()
    imgplot = single_time_frame.plot.imshow(ax=ax[idx], x="x", y="y", rgb="bands")
    imgplot.set_cmap('OrRd')
plt.show()